# Verify Space Motion Functionality

In [2]:
import astropy.units as u
from astropy.time import Time
from astropy.coordinates import SkyCoord

#exagerated motions for testing purposes
c = SkyCoord(l=10*u.degree, b=45*u.degree, distance=100*u.pc,
             pm_l_cosb=340*u.mas/u.yr, pm_b=-1170*u.mas/u.yr,
             frame='galactic',
             obstime=Time('1988-12-18 05:11:23.5'))

In [3]:
c

<SkyCoord (Galactic): (l, b, distance) in (deg, deg, pc)
    (10., 45., 100.)
 (pm_l_cosb, pm_b) in mas / yr
    (340., -1170.)>

In [4]:
dt = 10*u.year
c_new = c.apply_space_motion(dt=dt)
c_new

<SkyCoord (Galactic): (l, b, distance) in (deg, deg, pc)
    (10.00133557, 44.99674999, 99.99999448)
 (pm_l_cosb, pm_b, radial_velocity) in (mas / yr, mas / yr, km / s)
    (339.98071452, -1170.00559973, 0.03411744)>

In [5]:
import numpy as np
np.allclose(c_new.b, (c.b + c.pm_b*dt))

True

In [6]:
f'{c_new.b/ (c.b + c.pm_b*dt):.10f}'

'0.9999999998'

In [7]:
np.allclose(c_new.l, (c.l+(c.pm_l_cosb/np.cos(c.b))*dt))

True

`apply_space_motion` appears to work the way I think it should, that is:
\begin{equation}
\begin{align*}
\delta' &= \delta + \dot{\delta}dt \cr
\alpha' &= \alpha + \dot{\alpha}dt
\end{align*}
\end{equation}

# Verify Time Functionality

In [8]:
from astropy.time import Time
import astropy.units as u

# gaia ref_epoch on each gaia source record:
ref_epoch = 2016 #for dr3
# gaia doc specifies ref_epoch is tcb
t_gaia = Time(ref_epoch, scale='tcb',format='jyear')
t_gaia

<Time object: scale='tcb' format='jyear' value=2016.0>

In [9]:
t_gaia.iso

'2016-01-01 12:00:00.000'

In [10]:
#Fits header has OBSTIME as MJD
mjd = 53739.304355 # value from the header
t_obs = Time(mjd, scale='utc', format='mjd')
t_obs

<Time object: scale='utc' format='mjd' value=53739.304355>

In [11]:
t_obs.iso

'2006-01-04 07:18:16.272'

In [12]:
# calculate difference btwn the two times
#difference in seconds
(t_obs-t_gaia).sec


-315290419.4613611

In [13]:
#difference in years
(((t_gaia-t_obs).sec)*u.second).to(u.year)

<Quantity 9.99095065 yr>

In [14]:
c = SkyCoord(l=10*u.degree, b=45*u.degree, distance=100*u.pc,
             pm_l_cosb=340*u.mas/u.yr, pm_b=-1170*u.mas/u.yr,
             frame='galactic',
             obstime=t_gaia)
c_new = c.apply_space_motion(t_obs)

In [15]:
dt = t_obs-t_gaia
np.allclose(c_new.l, (c.l+(c.pm_l_cosb/np.cos(c.b))*dt))

True

In [16]:
np.allclose(c_new.b, (c.b + c.pm_b*dt))

True

Time and motions seem to check out!!

# Radial Velocity
Check three cases: rv=None, rv=np.nan, rv=something. None and nan should be same

In [17]:
import os, sys
sys.path.append(os.path.expanduser('~/repos/ReipurthBallyProject/src'))
sys.path.append(os.path.expanduser('~/repos/runawaysearch/src'))
from gaiastars import gaiastars as gs

In [18]:
gstars = gs(name='rv test', description='bunch of stars from random location')
gstars.conesearch(59.0*u.degree, 0.0*u.degree, 1.0*u.degree)
print(f'Number of stars returned: {len(gstars)}')
print(f'Number of stars missing Radial Velocity: {np.isnan(gstars.objs.radial_velocity).sum()}')

INFO: Query finished. [astroquery.utils.tap.core]
Number of stars returned: 2048
Number of stars missing Radial Velocity: 1437


In [19]:
objs = gstars.objs
#coords with no RV
coords_norv = SkyCoord(ra=np.array(objs.ra)*u.degree,
                    dec=np.array(objs.dec)*u.degree,
                    pm_ra_cosdec = np.array(objs.pmra)*u.mas/u.year,
                    pm_dec = np.array(objs.pmdec)*u.mas/u.year, obstime=t_gaia)

#stuff in rv = zero
coords_rv0 = SkyCoord(ra=np.array(objs.ra)*u.degree,
                    dec=np.array(objs.dec)*u.degree,
                    pm_ra_cosdec = np.array(objs.pmra)*u.mas/u.year,
                    pm_dec = np.array(objs.pmdec)*u.mas/u.year,
                    radial_velocity=np.zeros(len(objs), dtype=float)*u.km/u.second,obstime=t_gaia)

#apply space motion to the two sets of coords
coords_new_norv = coords_norv.apply_space_motion(new_obstime=t_obs)
coords_new_rv0 = coords_rv0.apply_space_motion(new_obstime=t_obs)

c:\Users\Kevin\anaconda3\envs\astro\lib\site-packages\erfa\core.py:154: ErfaWarning: ERFA function "pmsafe" yielded 2048 of "distance overridden (Note 6)"
  warnings.warn('ERFA function "{}" yielded {}'.format(func_name, wmsg),
c:\Users\Kevin\anaconda3\envs\astro\lib\site-packages\erfa\core.py:154: ErfaWarning: ERFA function "pmsafe" yielded 2047 of "distance overridden (Note 6)", 1 of "binary logical OR of the above warnings"
  warnings.warn('ERFA function "{}" yielded {}'.format(func_name, wmsg),


In [20]:
ra_diff = coords_new_norv.ra != coords_new_rv0.ra; dec_diff = coords_new_norv.dec != coords_new_rv0.dec
print(f'Number of RA differences: {ra_diff.sum()}')
print(f'Number of Dec Differences: {dec_diff.sum()}')
print(f'Number of either: {np.logical_or(ra_diff, dec_diff).sum()}')

Number of RA differences: 33
Number of Dec Differences: 37
Number of either: 37


In [21]:
with np.printoptions(precision=15, suppress=False):
    print((coords_new_norv.ra.value - coords_new_rv0.ra.value)[ra_diff])

[-1.464428578401566e-11 -2.818651978486741e-10  5.705658168153604e-11
 -1.483257960899209e-10 -2.520572195408022e-09 -6.040110633875884e-10
 -1.521563319784036e-09  5.976517059025355e-10  1.183764197776327e-10
 -1.729965504182474e-09  1.130615601141471e-10 -3.603943810048804e-10
 -3.515125968078792e-10 -1.867306309577543e-11  1.158184659288963e-12
  1.421085471520200e-14 -4.187512558928574e-10 -1.064393018168630e-11
  6.679812258880702e-10 -3.461764208623208e-11 -7.757705589028774e-11
 -6.437517185986508e-12 -5.412914561020443e-11  1.677804561950325e-10
  3.321929398225620e-10  5.717524231840798e-10 -1.757172185534728e-11
 -1.152820061633975e-09  1.111004621634493e-10  4.785505325344275e-11
 -2.223998762929114e-11 -4.938769393447728e-10 -1.519921966064430e-10]


In [22]:
with np.printoptions(precision=15, suppress=False):
    print((coords_new_norv.dec.value - coords_new_rv0.dec.value)[dec_diff])

[-2.010127619911373e-10 -1.205519173375080e-08 -1.189296772174941e-09
 -5.615448106510712e-10 -3.219960964528923e-09  2.845077506918869e-10
 -3.821105765133836e-09 -1.037887276122973e-09  8.269687157280714e-10
 -4.171740730640749e-10  9.437243875254353e-09 -1.730925847098774e-09
 -2.211213101510623e-09 -5.101322142486708e-10 -2.992573688853639e-10
  1.387778780781446e-16 -6.764515197987819e-10 -6.245004513516506e-17
  9.002639744792873e-10 -1.060775023375982e-09  1.852933362300746e-10
 -5.551115123125783e-17  7.512553745758765e-09 -3.687994437617803e-09
 -8.326672684688674e-17 -3.494920464142126e-10  2.168404344971009e-19
 -1.660803050640425e-09 -6.179091127656022e-09  3.605369558457028e-09
 -4.991740354398644e-10 -4.034280409737079e-09  1.404938165805447e-09
  1.788839631977623e-09  9.631579978020000e-10  8.436783494047972e-10
  1.938232574438814e-09]


### Conclusion
Comparing two sets of coordinates, one without RV specified, one with RV specified as 0, produced an exact match of position in 2011 out of 2048 cases or 98.2% of the cases. Of the 37 cases that result in different positions, the difference is on the order of $10^{-9}$ degree or 1 micro arc-second.

**Explicitly setting radial velocity to zero is equivalent to not specifying it**

**If radial velocity is not supplied, astropy assumes a value of zero**

has_rv = np.isfinite(gstars.objs.radial_velocity)
has_rv.sum()

In [23]:
has_rv = np.isfinite(gstars.objs.radial_velocity)
objs = gstars.objs.loc[has_rv]

coords_norv = SkyCoord(ra=np.array(objs.ra)*u.degree,
                    dec=np.array(objs.dec)*u.degree,
                    pm_ra_cosdec = np.array(objs.pmra)*u.mas/u.year,
                    pm_dec = np.array(objs.pmdec)*u.mas/u.year, obstime=t_gaia)

#use the rv returned from Gaia
coords_rv = SkyCoord(ra=np.array(objs.ra)*u.degree,
                    dec=np.array(objs.dec)*u.degree,
                    pm_ra_cosdec = np.array(objs.pmra)*u.mas/u.year,
                    pm_dec = np.array(objs.pmdec)*u.mas/u.year,
                    radial_velocity=np.array(objs.radial_velocity)*u.km/u.second,obstime=t_gaia)

#apply space motion to the two sets of coords
coords_new_norv = coords_norv.apply_space_motion(new_obstime=t_obs)
coords_new_rv = coords_rv.apply_space_motion(new_obstime=t_obs)

c:\Users\Kevin\anaconda3\envs\astro\lib\site-packages\erfa\core.py:154: ErfaWarning: ERFA function "pmsafe" yielded 611 of "distance overridden (Note 6)"
  warnings.warn('ERFA function "{}" yielded {}'.format(func_name, wmsg),


In [24]:
ra_diff = coords_new_norv.ra != coords_new_rv.ra; dec_diff = coords_new_norv.dec != coords_new_rv.dec
print(f'Number of RA differences: {ra_diff.sum()}')
print(f'Number of Dec Differences: {dec_diff.sum()}')
print(f'Number of either: {np.logical_or(ra_diff, dec_diff).sum()}')

Number of RA differences: 518
Number of Dec Differences: 608
Number of either: 609


In [25]:
np.isclose(coords_new_rv.dec,coords_new_norv.dec, rtol=1e-9).sum()

610

### Conclusion
The inclusion of the radial velocity parameter in the application of space motion makes a difference, albeit a small difference, in the resulting positions from `apply_space_motion`, at least in the relatively short time deltas we are using.

## Coordinate Frames

FK4
    : Fundamental Katalogue 4

FK5
    : Fundamental Katlogue 5


ICRS
    : International Celestial Reference System

ICRF
    : International Celestial Reference Frame (a specific realization of the ICRS)

[USNO Article ICRS](https://aa.usno.navy.mil/faq/ICRS_doc)
> The ICRS axes are consistent, to better than 0.1 arcsecond, with the equator and equinox of J2000.0 defined by the dynamics of the Earth. However, the ICRS axes are meant to be regarded as fixed directions in space that have an existence independent of the dynamics of the Earth or the particular set of objects used to define them at any given time.

$\implies$ ICRS and FK5 cooodinates nearly equivalent for the same epoch.

$\implies$ Don't need an equinox with ICRS since axis are fixed wrt. distant quasars, but an epoch is needed, to fix the point in time when an object was at its coordinates.

In [26]:
from astropy.coordinates import ICRS, FK4, FK5
c = SkyCoord(ra=0*u.degree, dec=0*u.degree)
c

<SkyCoord (ICRS): (ra, dec) in deg
    (0., 0.)>

In [27]:
c.transform_to(FK4(equinox='J2000.0')),c.transform_to(FK4()),c.transform_to(FK5())

(<SkyCoord (FK4: equinox=J2000.000, obstime=J2000.000): (ra, dec) in deg
     (359.99966387, -6.04472246e-06)>,
 <SkyCoord (FK4: equinox=B1950.000, obstime=B1950.000): (ra, dec) in deg
     (359.35931432, -0.27840736)>,
 <SkyCoord (FK5: equinox=J2000.000): (ra, dec) in deg
     (6.36111135e-06, 2.52777716e-06)>)

### DS9
1. load fits file, zoom 'fit'
1. set wcs to degrees
2. View header, find and note CRPIX1 and CRPIX2, CRVAL1 and CRVAL2
3. Analysis $\rightarrow$ Cross Hair Parameters, note coordinate system ICRS, FK4 or FK5
4. Select image coordinates, paste in values for CRPIX1 and CRPIX2
5. Go back to WCS (in Cross Hair Parameters pane), select alternately ICRS, FK4 and FK5
6. Compare to CRVAL1 and CRVAL2

In [28]:
import os
from astropy.io import fits
ds9 = {}
files = ['hh34_ha', 'hh34_sii', 'ZCMa_ha']
for file in files:
    fitspath = os.path.join('../data', file+'.fits')
    with fits.open(fitspath) as hdul:
        hdu = hdul[0]
        hdr = hdu.header
        ds9[file] = {'hdr':{'CRPIX1':hdr['CRPIX1'],
                            'CRPIX2':hdr['CRPIX2'],
                            'CRVAL1':hdr['CRVAL1'],
                            'CRVAL2':hdr['CRVAL2'] }
                     }


In [29]:
# values pasted from DS9:
ds9['hh34_ha']['fk4']={'ra':83.9079312, 'dec':-6.4845724}
ds9['hh34_ha']['fk5']={'ra':84.5171394, 'dec':-6.4564928}
ds9['hh34_ha']['icrs']={'ra':84.5171321, 'dec':-6.4564869}

ds9['hh34_sii']['fk4']={'ra':83.9079312, 'dec':-6.4845724}
ds9['hh34_sii']['fk5']={'ra':84.5171394, 'dec':-6.4564928}
ds9['hh34_sii']['icrs']={'ra':84.5171321, 'dec':-6.4564869}

ds9['ZCMa_ha']['fk4']={'ra':105.9576268, 'dec':-11.4665181}
ds9['ZCMa_ha']['fk5']={'ra':106.5438437, 'dec':-11.5444012}
ds9['ZCMa_ha']['icrs']={'ra':106.5438361,'dec':-11.5443950}

ds9

{'hh34_ha': {'hdr': {'CRPIX1': 5717.0,
   'CRPIX2': 4499.0,
   'CRVAL1': 83.9079311590073,
   'CRVAL2': -6.48457244114713},
  'fk4': {'ra': 83.9079312, 'dec': -6.4845724},
  'fk5': {'ra': 84.5171394, 'dec': -6.4564928},
  'icrs': {'ra': 84.5171321, 'dec': -6.4564869}},
 'hh34_sii': {'hdr': {'CRPIX1': 5728.0,
   'CRPIX2': 4510.0,
   'CRVAL1': 83.9079311590073,
   'CRVAL2': -6.48457244114713},
  'fk4': {'ra': 83.9079312, 'dec': -6.4845724},
  'fk5': {'ra': 84.5171394, 'dec': -6.4564928},
  'icrs': {'ra': 84.5171321, 'dec': -6.4564869}},
 'ZCMa_ha': {'hdr': {'CRPIX1': 5715.0,
   'CRPIX2': 4501.0,
   'CRVAL1': 105.95762677501,
   'CRVAL2': -11.466518082996},
  'fk4': {'ra': 105.9576268, 'dec': -11.4665181},
  'fk5': {'ra': 106.5438437, 'dec': -11.5444012},
  'icrs': {'ra': 106.5438361, 'dec': -11.544395}}}

In [30]:
simbad = {'hh34': SkyCoord.from_name('HH34'), 'ZCMa': SkyCoord.from_name('Z CMa'),
           'hh160': SkyCoord.from_name('HH 160')} #, 'hh388':SkyCoord.from_name('HH 388')}


In [31]:
simbad

{'hh34': <SkyCoord (ICRS): (ra, dec) in deg
     (83.880417, -6.478611)>,
 'ZCMa': <SkyCoord (ICRS): (ra, dec) in deg
     (105.92983309, -11.55172474)>,
 'hh160': <SkyCoord (ICRS): (ra, dec) in deg
     (105.92958, -11.55167)>}

Set DS9 cross-hair parameters to WCS+fk4 and pasted the **fk5** values below into them.

In [32]:
simbad['ZCMa'].fk5,  simbad['hh160'].fk5

(<SkyCoord (FK5: equinox=J2000.000): (ra, dec) in deg
     (105.92983926, -11.55173075)>,
 <SkyCoord (FK5: equinox=J2000.000): (ra, dec) in deg
     (105.92958617, -11.55167601)>)

In [33]:
simbad['hh34'].fk5

<SkyCoord (FK5: equinox=J2000.000): (ra, dec) in deg
    (83.88042301, -6.47861623)>

## Notes and Summary

It appears that:
+ DS9 in absence of a RADESYS card defaults to FK4 coordinates
+ Bo's two images (hh34 and ZMCa) don't have the RADESYS cards, therefore DS9 treats them as FK4
+ SIMBAD queries return objects in ICRS coords. Reframing them as FK5 and then  **entering them as if they were FK4 coordinates** places the object directly on top of object in image
+ Gaia catalog search (from within DS9) will misplace objects returned
+ Should not make a difference when using astropy `WCS` class and methods as the `WCS` operations all use the ICRS frame.

## DS9 Gaia Catalog Query

What exactly does DS9 do with query results back Gaia?

In [33]:
from astropy.table import Table
t = Table.read('../data/ds9.xml')
t

_RAJ2000,_DEJ2000,RA_ICRS,e_RA_ICRS,DE_ICRS,e_DE_ICRS,Source,Plx,e_Plx,pmRA,e_pmRA,pmDE,e_pmDE,Dup,FG,e_FG,Gmag,e_Gmag,FBP,e_FBP,BPmag,e_BPmag,FRP,e_FRP,RPmag,e_RPmag,BP-RP,RV,e_RV,Teff,AG,E_BP-RP_,Rad,Lum
deg,deg,deg,mas,deg,mas,,mas,mas,mas / yr,mas / yr,mas / yr,mas / yr,,,,mag,mag,,,mag,mag,,,mag,mag,mag,km / s,km / s,K,mag,mag,Rsun,Lsun
float64,float64,float64,float64,float64,float64,int64,float64,float32,float64,float32,float64,float32,uint8,float32,float32,float64,float64,float32,float32,float64,float64,float32,float32,float64,float64,float64,float64,float32,float64,float32,float32,float32,float64
106.5960297972769,-11.8231343181156,106.59602990285,0.1000,-11.82313082631,0.1192,3045765784397960320,0.0653,0.1384,0.024,0.205,0.811,0.203,0,1502,2.135,17.7468,0.0015,459,6.338,18.6968,0.0150,1591,10.82,16.7575,0.0074,1.9393,--,--,--,--,--,--,--
106.5874708290363,-11.8203845869555,106.58746673812,0.2509,-11.82038400140,0.2502,3045765784397962880,0.2677,0.3062,-0.930,0.549,0.136,0.539,0,455.1,1.623,19.0430,0.0039,202.6,10.36,19.5850,0.0556,371.8,10.39,18.3363,0.0303,1.2488,--,--,--,--,--,--,--
106.5862601488865,-11.8228688876411,106.58625837614,0.3232,-11.82286709653,0.3217,3045765784397960832,0.0177,0.3959,-0.403,0.684,0.416,0.723,0,320.2,1.494,19.4248,0.0051,119.7,13.12,20.1562,0.1190,295.5,8.54,18.5855,0.0314,1.5707,--,--,--,--,--,--,--
106.5848990215625,-11.8194283374477,106.58489292040,0.1028,-11.81942157342,0.1069,3045765784397963264,0.1981,0.1258,-1.387,0.209,1.571,0.195,0,1502,2.154,17.7469,0.0016,654.2,10.45,18.3122,0.0173,1275,12.76,16.9979,0.0109,1.3143,--,--,--,--,--,--,--
106.5919793421407,-11.8229065547577,106.59197527758,0.0869,-11.82290126323,0.0942,3045765788700116608,0.2312,0.1093,-0.924,0.177,1.229,0.168,0,1939,2.368,17.4695,0.0013,878,11.11,17.9927,0.0137,1618,18.99,16.7391,0.0127,1.2536,--,--,--,--,--,--,--
106.5886225278467,-11.8226691935784,106.58860571536,1.1089,-11.82267648719,1.1171,3045765788696951424,-0.8427,1.5955,-3.822,2.156,-1.694,2.245,0,80.95,1.282,20.9178,0.0172,52.33,10.48,21.0546,0.2175,92.98,7.228,19.8410,0.0844,1.2136,--,--,--,--,--,--,--
106.6015895994260,-11.8074022893954,106.60154511172,0.0212,-11.80735947926,0.0227,3045765891779315200,1.0773,0.0260,-10.114,0.042,9.943,0.040,0,5.28e+04,13.72,13.8817,0.0003,2.424e+04,42.41,14.3902,0.0019,4.088e+04,38.19,13.2332,0.0010,1.1570,--,--,5035.33,--,--,1.84,1.969
106.6046863355600,-11.8072176037700,106.60468633556,1.8280,-11.80721760377,2.4950,3045765926135885056,--,--,--,--,--,--,0,76.12,1.467,20.9846,0.0209,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--


In [34]:
from astropy.time import Time
import astropy.units as u

# gaia ref_epoch on each gaia source record:
ref_epoch = 2015.5 #for dr2
# gaia doc specifies ref_epoch is tcb
t_gaia = Time(ref_epoch, scale='tcb',format='jyear')
t_gaia, t_gaia.iso

(<Time object: scale='tcb' format='jyear' value=2015.5>,
 '2015-07-02 21:00:00.000')

In [35]:
# get the MJD from the fits file

fitspath = os.path.join('../data', 'ZCMa_ha.fits')
with fits.open(fitspath) as hdul:
    hdu = hdul[0]
    hdr = hdu.header
    mjd = hdr['MJD'] # value from the header

t_obs = Time(mjd, scale='utc', format='mjd')
t_obs.iso

'2006-01-04 10:40:44.976'

In [36]:
# make the coordinates from the ICRS values and the GAIA ref epoch
coords_icrs = SkyCoord(ra=t['RA_ICRS'],
                dec=t['DE_ICRS'],
                pm_ra_cosdec=t['pmRA'].filled(fill_value=0.0),
                pm_dec=t['pmDE'].filled(fill_value=0.0), obstime=t_gaia)

In [37]:
# function to compute root mean squared error
def rmse(hat, act):
    e2 = (hat-act)**2
    return np.sqrt(np.nanmean(e2))

Compare five predictions of the _RAJ2000 and _DEJ2000 values:
1. with the ICRS values
2. with the ICRS values transformed to FK5
3. with the ICRS values, applied space motion to the observation date, transformed to FK5
3. ICRS values, applied space motion to J2000, transformed to FK5
3. ICRS values applied space motion to J2000

In [38]:
# the dependent variable
coords_j2000 = SkyCoord(ra=t['_RAJ2000'],
                dec=t['_DEJ2000'])

# case 1: icrs as predictor
ra_rmse_icrs = rmse(coords_icrs.ra, coords_j2000.ra).to(u.mas)
dec_rmse_icrs = rmse(coords_icrs.dec, coords_j2000.dec).to(u.mas)
print(f'ICRS RMSE: RA: {ra_rmse_icrs}, Dec: {dec_rmse_icrs}')

#case 3: fk5 as predictor
coords_fk5 = coords_icrs.fk5
ra_rmse_fk5 = rmse(coords_fk5.ra, coords_j2000.ra).to(u.mas)
dec_rmse_fk5 = rmse(coords_fk5.dec, coords_j2000.dec).to(u.mas)
print(f'FK5 RMSE: RA: {ra_rmse_fk5}, Dec: {dec_rmse_fk5}')

# case 3: icrs moved to obs date, then reframed to fk5
coords_fk5_m = coords_icrs.apply_space_motion(new_obstime=t_obs).fk5
ra_rmse_fk5_m = rmse(coords_fk5_m.ra, coords_j2000.ra).to(u.mas)
dec_rmse_fk5_m = rmse(coords_fk5_m.dec, coords_j2000.dec).to(u.mas)
print(f'FK5_obs RMSE: RA: {ra_rmse_fk5_m}, Dec: {dec_rmse_fk5_m}')

# case 4: icrs moved to J2000 date, then reframed to fk5
t_2000 = Time('J2000.0',  format='jyear_str')
coords_fk5_m2000 = coords_icrs.apply_space_motion(new_obstime=t_2000).fk5
ra_rmse_fk5_m2000 = rmse(coords_fk5_m2000.ra, coords_j2000.ra).to(u.mas)
dec_rmse_fk5_m2000 = rmse(coords_fk5_m2000.dec, coords_j2000.dec).to(u.mas)
print(f'FK5_2000 RMSE: RA: {ra_rmse_fk5_m2000}, Dec: {dec_rmse_fk5_m2000}')
t_2000 = Time('J2000.0',  format='jyear_str')

# case 5: icrs moved to J2000 date
coords_icrs_m2000 = coords_icrs.apply_space_motion(new_obstime=t_2000)
ra_rmse_icrs_m2000 = rmse(coords_icrs_m2000.ra, coords_j2000.ra).to(u.mas)
dec_rmse_icrs_m2000 = rmse(coords_icrs_m2000.dec, coords_j2000.dec).to(u.mas)
print(f'ICRS_2000 RMSE: RA: {ra_rmse_icrs_m2000}, Dec: {dec_rmse_icrs_m2000}')

ICRS RMSE: RA: 46.20837588028197 mas, Dec: 61.91913934382889 mas
FK5 RMSE: RA: 45.91363525102139 mas, Dec: 63.48592363332249 mas
FK5_obs RMSE: RA: 24.78335912811097 mas, Dec: 30.6590319712022 mas
FK5_2000 RMSE: RA: 22.27391177861022 mas, Dec: 21.667139300147635 mas
ICRS_2000 RMSE: RA: 0.0010385274308936654 mas, Dec: 0.0020881441451843625 mas


c:\Users\Kevin\anaconda3\envs\astro\lib\site-packages\erfa\core.py:154: ErfaWarning: ERFA function "pmsafe" yielded 13562 of "distance overridden (Note 6)", 3 of "binary logical OR of the above warnings"
  warnings.warn('ERFA function "{}" yielded {}'.format(func_name, wmsg),


In [39]:
res_table = Table(rows=[('ICRS', ra_rmse_icrs, dec_rmse_icrs),
                        ('FK5', ra_rmse_fk5, dec_rmse_fk5),
                        ('FK5 Obs',ra_rmse_fk5_m, dec_rmse_fk5_m),
                        ('FK5_2000', ra_rmse_fk5_m2000, dec_rmse_fk5_m2000),
                        ('ICRS_2000', ra_rmse_icrs_m2000, dec_rmse_icrs_m2000)],
                        names=['Predictor', 'RMSE_RA', 'RMSE_DEC'])
res_table['RMSE_RA'].format = '.3f'
res_table['RMSE_DEC'].format = '.3f'

print(f'Number of observations: {len(coords_icrs)}')

res_table

Number of observations: 13565


Predictor,RMSE_RA,RMSE_DEC
,mas,mas
str9,float64,float64
ICRS,46.208,61.919
FK5,45.914,63.486
FK5 Obs,24.783,30.659
FK5_2000,22.274,21.667
ICRS_2000,0.001,0.002


**Conclusion** DS9 takes the returned values from Gaia and moves them back to epoch J2000.